# Proyecto de Clasificación Multiclase: Clasificación Estelar

**Curso:** Inteligencia Artificial  
**Tema:** Clasificación Multiclase con Alta Dimensionalidad  
**Dataset:** Stellar Classification Dataset - SDSS17

---

## Objetivo del Proyecto

Este proyecto tiene como objetivo desarrollar un modelo de clasificación multiclase para identificar objetos astronómicos (estrellas, galaxias y cuásares) utilizando datos del Sloan Digital Sky Survey (SDSS). El proyecto incluye:

1. **Análisis Exploratorio de Datos (EDA)** completo
2. **Preprocesamiento** de datos (limpieza, escalado, balanceo)
3. **Reducción de dimensionalidad** usando PCA
4. **Entrenamiento y comparación** de múltiples modelos de Machine Learning
5. **Calibración de probabilidades** para mejorar la confianza de las predicciones
6. **Interpretabilidad** del modelo para entender qué características son más importantes
7. **Análisis de robustez** para evaluar la generalización del modelo
8. **Conclusiones y recomendaciones** para posible uso en producción

---

## 0. Configuración del Entorno

En esta sección importamos todas las librerías necesarias y verificamos las versiones de las principales herramientas que vamos a utilizar.

In [ ]:
# Librerías básicas para manipulación de datos
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Librerías de scikit-learn para preprocesamiento
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif

# Librerías para reducción de dimensionalidad
from sklearn.decomposition import PCA

# Modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Calibración de modelos
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

# Métricas de evaluación
from sklearn.metrics import (
    accuracy_score, 
    balanced_accuracy_score,
    precision_score, 
    recall_score, 
    f1_score,
    classification_report,
    confusion_matrix,
    brier_score_loss,
    ConfusionMatrixDisplay
)

# Configurar semilla aleatoria para reproducibilidad
SEED = 42
np.random.seed(SEED)

# Mostrar versiones de las librerías principales
print("="*60)
print("VERSIONES DE LIBRERÍAS")
print("="*60)
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")
import sklearn
print(f"Scikit-learn: {sklearn.__version__}")
print("="*60)

---

## 1. Carga de Datos y Análisis Exploratorio (EDA)

En esta sección vamos a:
- Cargar el dataset desde el archivo CSV
- Explorar la estructura de los datos
- Analizar la distribución de las variables
- Identificar valores faltantes
- Visualizar correlaciones entre variables
- Detectar características con poca varianza

### 1.1 Carga del Dataset

In [ ]:
# Cargar el dataset desde el archivo CSV
# Ruta del archivo: ajustar si es necesario
ruta_dataset = r"Data\archive\star_classification.csv"

# Leer el archivo CSV
df = pd.read_csv(ruta_dataset)

print("Dataset cargado exitosamente.")
print(f"Dimensiones del dataset: {df.shape[0]} filas y {df.shape[1]} columnas")

### 1.2 Exploración Inicial del Dataset

In [ ]:
# Mostrar las primeras filas del dataset para entender su estructura
print("Primeras 5 filas del dataset:")
print("="*80)
df.head()

In [ ]:
# Información general del dataset: tipos de datos, valores no nulos
print("Información general del dataset:")
print("="*80)
df.info()

In [ ]:
# Verificar valores faltantes por columna
print("Valores faltantes por columna:")
print("="*80)
valores_faltantes = df.isnull().sum()
porcentaje_faltantes = (valores_faltantes / len(df)) * 100

# Crear un DataFrame para mostrar la información de manera clara
df_faltantes = pd.DataFrame({
    'Columna': valores_faltantes.index,
    'Valores Faltantes': valores_faltantes.values,
    'Porcentaje (%)': porcentaje_faltantes.values
})

# Filtrar solo las columnas con valores faltantes
df_faltantes = df_faltantes[df_faltantes['Valores Faltantes'] > 0]

if len(df_faltantes) > 0:
    print(df_faltantes.to_string(index=False))
else:
    print("No hay valores faltantes en el dataset.")

### 1.3 Análisis de la Variable Objetivo

La variable objetivo es `class`, que indica el tipo de objeto astronómico:
- **GALAXY**: Galaxia
- **STAR**: Estrella
- **QSO**: Cuásar (Quasi-Stellar Object)

Vamos a analizar la distribución de estas clases para identificar posibles desbalances.

In [ ]:
# Contar la frecuencia de cada clase
print("Distribución de la variable objetivo 'class':")
print("="*80)
conteo_clases = df['class'].value_counts()
porcentaje_clases = (conteo_clases / len(df)) * 100

# Crear DataFrame para mostrar la distribución
df_distribucion = pd.DataFrame({
    'Clase': conteo_clases.index,
    'Frecuencia': conteo_clases.values,
    'Porcentaje (%)': porcentaje_clases.values
})

print(df_distribucion.to_string(index=False))

In [ ]:
# Visualizar la distribución de clases con un gráfico de barras
plt.figure(figsize=(10, 6))
ax = sns.countplot(data=df, x='class', order=df['class'].value_counts().index)
plt.title('Distribución de Clases en el Dataset', fontsize=16, fontweight='bold')
plt.xlabel('Clase', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)

# Añadir valores sobre las barras
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2., height,
            f'{int(height)}\n({height/len(df)*100:.1f}%)',
            ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Comentario sobre el desbalance
print("\n📊 Observación:")
if df_distribucion['Porcentaje (%)'].max() > 50:
    print("El dataset presenta un desbalance de clases. La clase mayoritaria representa")
    print(f"más del 50% de las observaciones. Esto puede afectar el desempeño del modelo.")
    print("Consideraremos técnicas de balanceo o ajuste de pesos de clase durante el entrenamiento.")
else:
    print("El dataset tiene una distribución relativamente balanceada entre clases.")

### 1.4 Estadísticas Descriptivas de Variables Numéricas

In [ ]:
# Mostrar estadísticas descriptivas de las variables numéricas
print("Estadísticas descriptivas de variables numéricas:")
print("="*80)
df.describe().T

In [ ]:
# Identificar columnas numéricas relevantes para el análisis
# Excluiremos IDs y metadatos técnicos que no aportan información predictiva
columnas_a_excluir = ['obj_ID', 'run_ID', 'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'plate', 'MJD', 'fiber_ID']

# Obtener todas las columnas numéricas excepto las excluidas
columnas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
columnas_caracteristicas = [col for col in columnas_numericas if col not in columnas_a_excluir]

print(f"\nColumnas numéricas que se usarán como características: {columnas_caracteristicas}")
print(f"Total de características numéricas: {len(columnas_caracteristicas)}")

### 1.5 Análisis de Correlación

Vamos a calcular y visualizar la matriz de correlación entre las variables numéricas para identificar:
- Variables altamente correlacionadas (redundancia)
- Patrones de relación entre características
- Posibles problemas de multicolinealidad

In [ ]:
# Calcular la matriz de correlación
matriz_correlacion = df[columnas_caracteristicas].corr()

# Visualizar la matriz de correlación con un mapa de calor
plt.figure(figsize=(12, 10))
sns.heatmap(matriz_correlacion, 
            annot=True, 
            fmt='.2f', 
            cmap='coolwarm', 
            center=0,
            square=True,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación de Características Numéricas', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📊 Observación:")
print("Las variables de magnitud (u, g, r, i, z) pueden estar correlacionadas entre sí,")
print("lo cual es esperado en datos astronómicos. Esto justifica el uso de PCA para")
print("reducir dimensionalidad y eliminar redundancia.")

### 1.6 Detección de Características con Varianza Baja

Las características con varianza muy baja o nula no aportan información útil para la clasificación. Vamos a identificarlas y decidir si eliminarlas.

In [ ]:
# Calcular la varianza de cada característica numérica
varianzas = df[columnas_caracteristicas].var()

print("Varianza de cada característica:")
print("="*80)
print(varianzas.sort_values())

# Definir un umbral para varianza baja (por ejemplo, 0.01)
umbral_varianza = 0.01

# Identificar características con varianza menor al umbral
caracteristicas_baja_varianza = varianzas[varianzas < umbral_varianza]

print(f"\n\nCaracterísticas con varianza menor a {umbral_varianza}:")
if len(caracteristicas_baja_varianza) > 0:
    print(caracteristicas_baja_varianza)
    print(f"\nSe recomienda eliminar estas {len(caracteristicas_baja_varianza)} característica(s).")
else:
    print("No se encontraron características con varianza extremadamente baja.")
    print("Todas las características tienen varianza suficiente para ser consideradas.")

### 1.7 Visualización de Distribuciones por Clase

Vamos a visualizar cómo se distribuyen algunas de las características principales según la clase del objeto astronómico.

In [ ]:
# Seleccionar algunas características clave para visualizar
caracteristicas_visualizar = ['u', 'g', 'r', 'i', 'z', 'redshift']

# Crear subplots para visualizar distribuciones
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, caracteristica in enumerate(caracteristicas_visualizar):
    for clase in df['class'].unique():
        datos_clase = df[df['class'] == clase][caracteristica]
        axes[idx].hist(datos_clase, alpha=0.6, label=clase, bins=30)
    
    axes[idx].set_xlabel(caracteristica, fontsize=10)
    axes[idx].set_ylabel('Frecuencia', fontsize=10)
    axes[idx].set_title(f'Distribución de {caracteristica} por Clase', fontsize=11, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Observación:")
print("Las diferentes clases muestran distribuciones distintas en las características,")
print("lo que sugiere que estas variables tienen poder discriminativo para la clasificación.")

### 1.8 Resumen del EDA

**Conclusiones del Análisis Exploratorio:**

1. **Tamaño del Dataset**: El dataset contiene información de múltiples objetos astronómicos con características numéricas.

2. **Variable Objetivo**: Tenemos 3 clases (GALAXY, STAR, QSO). La distribución puede estar desbalanceada, lo que consideraremos en el preprocesamiento.

3. **Características**: Las principales características son las magnitudes en diferentes bandas (u, g, r, i, z), coordenadas (alpha, delta) y redshift.

4. **Correlaciones**: Existe correlación entre algunas variables, especialmente entre las magnitudes, lo que justifica el uso de PCA.

5. **Valores Faltantes**: Se identificaron (o no) valores faltantes que deberán ser tratados en el preprocesamiento.

6. **Separabilidad**: Las distribuciones de características por clase sugieren que es posible discriminar entre las diferentes categorías.

---

## 2. Preprocesamiento de Datos

En esta sección vamos a:
- Separar características (X) y variable objetivo (y)
- Eliminar columnas irrelevantes (IDs, metadatos)
- Tratar valores faltantes mediante imputación
- Codificar la variable objetivo
- Analizar el desbalance de clases
- Escalar las características numéricas

### 2.1 Separación de Características y Variable Objetivo

In [ ]:
# Crear una copia del dataset original para no modificarlo
df_procesado = df.copy()

# Separar la variable objetivo (y) del resto de características
y = df_procesado['class']

# Seleccionar solo las características numéricas relevantes (X)
# Eliminamos IDs y metadatos que no aportan información predictiva
X = df_procesado[columnas_caracteristicas]

print(f"Dimensiones de X (características): {X.shape}")
print(f"Dimensiones de y (variable objetivo): {y.shape}")
print(f"\nCaracterísticas seleccionadas: {list(X.columns)}")

### 2.2 Tratamiento de Valores Faltantes

**Estrategia de imputación:**
- Para características numéricas, utilizaremos `SimpleImputer` con la **mediana**
- Elegimos la mediana porque es más robusta a valores atípicos (outliers) que la media
- Esto es importante en datos astronómicos que pueden tener mediciones extremas

In [ ]:
# Verificar si hay valores faltantes en X
print("Valores faltantes en X antes de la imputación:")
print(X.isnull().sum())

# Crear el imputador con estrategia de mediana
imputador = SimpleImputer(strategy='median')

# Ajustar y transformar los datos
X_imputado = imputador.fit_transform(X)

# Convertir de nuevo a DataFrame para mantener los nombres de las columnas
X_imputado = pd.DataFrame(X_imputado, columns=X.columns, index=X.index)

print("\nValores faltantes en X después de la imputación:")
print(X_imputado.isnull().sum())
print("\n✓ Imputación completada exitosamente.")

### 2.3 Codificación de la Variable Objetivo

Necesitamos convertir las etiquetas de texto (GALAXY, STAR, QSO) a valores numéricos para poder entrenar los modelos.

In [ ]:
# Crear el codificador de etiquetas
codificador_etiquetas = LabelEncoder()

# Ajustar y transformar las etiquetas
y_codificado = codificador_etiquetas.fit_transform(y)

# Mostrar la correspondencia entre etiquetas originales y codificadas
print("Correspondencia de etiquetas:")
print("="*40)
for i, clase in enumerate(codificador_etiquetas.classes_):
    print(f"{clase} → {i}")

# Guardar las clases originales para uso posterior
clases_originales = codificador_etiquetas.classes_
print(f"\n✓ Codificación completada. Total de clases: {len(clases_originales)}")

### 2.4 Análisis de Desbalance de Clases

El desbalance de clases puede causar que el modelo favorezca la clase mayoritaria. Vamos a calcular la distribución y considerar estrategias de balanceo.

In [ ]:
# Calcular la distribución de clases en términos de frecuencia
unique, counts = np.unique(y_codificado, return_counts=True)
distribucion_clases = dict(zip(unique, counts))

print("Distribución de clases (codificadas):")
print("="*40)
for clase_num, conteo in distribucion_clases.items():
    clase_nombre = clases_originales[clase_num]
    porcentaje = (conteo / len(y_codificado)) * 100
    print(f"Clase {clase_num} ({clase_nombre}): {conteo} ({porcentaje:.2f}%)")

# Calcular el ratio de desbalance
max_clase = max(counts)
min_clase = min(counts)
ratio_desbalance = max_clase / min_clase

print(f"\nRatio de desbalance (max/min): {ratio_desbalance:.2f}")

# Determinar estrategia de balanceo
if ratio_desbalance > 3:
    print("\n⚠️ El dataset presenta un desbalance significativo.")
    print("Estrategia recomendada:")
    print("  1. Usar 'class_weight=balanced' en los modelos que lo soporten")
    print("  2. Considerar técnicas de sobremuestreo (SMOTE) o submuestreo")
    print("  3. Usar métricas como F1-score macro y Balanced Accuracy")
    usar_class_weight = True
else:
    print("\n✓ El dataset tiene un desbalance moderado o está balanceado.")
    print("No es crítico usar técnicas de balanceo, pero consideraremos class_weight.")
    usar_class_weight = True

### 2.5 Escalado de Características

**¿Por qué escalar?**
- Muchos algoritmos (SVM, KNN, redes neuronales, PCA) son sensibles a la escala de las características
- El escalado asegura que todas las variables tengan la misma importancia inicial

**Elección del escalador:**
- **StandardScaler**: Estandariza las características para tener media 0 y desviación estándar 1
- **RobustScaler**: Alternativa más robusta a outliers (usa mediana y rango intercuartil)

Usaremos `StandardScaler` por defecto, pero comentaremos cuándo usar `RobustScaler`.

In [ ]:
# Crear el escalador estándar
escalador = StandardScaler()

# Ajustar el escalador con los datos y transformar
X_escalado = escalador.fit_transform(X_imputado)

# Convertir de nuevo a DataFrame para mantener los nombres de columnas
X_escalado = pd.DataFrame(X_escalado, columns=X.columns, index=X.index)

print("✓ Escalado completado exitosamente.")
print("\nEstadísticas después del escalado:")
print("="*60)
print("Medias (deben estar cerca de 0):")
print(X_escalado.mean())
print("\nDesviaciones estándar (deben estar cerca de 1):")
print(X_escalado.std())

### 2.6 Resumen del Preprocesamiento

**Pasos completados:**

1. ✓ Separación de características (X) y variable objetivo (y)
2. ✓ Eliminación de columnas irrelevantes (IDs y metadatos)
3. ✓ Imputación de valores faltantes usando la mediana
4. ✓ Codificación de la variable objetivo (texto → números)
5. ✓ Análisis de desbalance de clases y definición de estrategia
6. ✓ Escalado de características usando StandardScaler

**Datos listos para:**
- Reducción de dimensionalidad (PCA)
- División en conjuntos de entrenamiento y prueba
- Entrenamiento de modelos

---

## 3. Reducción de Dimensionalidad con PCA

El Análisis de Componentes Principales (PCA) nos permite:
- Reducir la dimensionalidad del dataset
- Eliminar redundancia entre características correlacionadas
- Mejorar la eficiencia computacional
- Facilitar la visualización de los datos
- Potencialmente mejorar el desempeño de algunos modelos

**Objetivos:**
1. Aplicar PCA sobre los datos escalados
2. Analizar la varianza explicada por componente
3. Elegir el número óptimo de componentes
4. Visualizar los datos en 2D usando las primeras componentes

### 3.1 Aplicación de PCA

In [ ]:
# Primero, aplicamos PCA con todas las componentes posibles para analizar la varianza
pca_completo = PCA(random_state=SEED)
pca_completo.fit(X_escalado)

# Obtener la varianza explicada por cada componente
varianza_explicada = pca_completo.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)

print("Varianza explicada por cada componente principal:")
print("="*60)
for i, (var_individual, var_acum) in enumerate(zip(varianza_explicada, varianza_acumulada)):
    print(f"PC{i+1}: {var_individual*100:.2f}% | Acumulada: {var_acum*100:.2f}%")

### 3.2 Visualización de la Varianza Explicada (Gráfico del Codo)

In [ ]:
# Crear figura con dos subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Varianza explicada por componente individual
ax1.bar(range(1, len(varianza_explicada) + 1), varianza_explicada * 100)
ax1.set_xlabel('Componente Principal', fontsize=12)
ax1.set_ylabel('Varianza Explicada (%)', fontsize=12)
ax1.set_title('Varianza Explicada por Componente Individual', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Subplot 2: Varianza explicada acumulada
ax2.plot(range(1, len(varianza_acumulada) + 1), varianza_acumulada * 100, 'o-', linewidth=2, markersize=8)
ax2.axhline(y=90, color='r', linestyle='--', label='90% varianza')
ax2.axhline(y=95, color='g', linestyle='--', label='95% varianza')
ax2.set_xlabel('Número de Componentes', fontsize=12)
ax2.set_ylabel('Varianza Explicada Acumulada (%)', fontsize=12)
ax2.set_title('Varianza Explicada Acumulada (Gráfico del Codo)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.3 Selección del Número Óptimo de Componentes

**Criterio:** Seleccionamos el número mínimo de componentes que expliquen al menos el 95% de la varianza total.

**Justificación:** 
- Mantener el 95% de la varianza asegura que conservamos la mayor parte de la información
- Al mismo tiempo, reducimos significativamente la dimensionalidad
- Esto mejora la eficiencia computacional sin sacrificar mucha información

In [ ]:
# Determinar el número de componentes necesarios para explicar el 95% de la varianza
umbral_varianza = 0.95
n_componentes_95 = np.argmax(varianza_acumulada >= umbral_varianza) + 1

print(f"Número de componentes para {umbral_varianza*100}% de varianza: {n_componentes_95}")
print(f"Varianza explicada con {n_componentes_95} componentes: {varianza_acumulada[n_componentes_95-1]*100:.2f}%")

# También mostrar para 90% y 99% como referencia
n_componentes_90 = np.argmax(varianza_acumulada >= 0.90) + 1
n_componentes_99 = np.argmax(varianza_acumulada >= 0.99) + 1

print(f"\nComparación:")
print(f"  - 90% varianza: {n_componentes_90} componentes")
print(f"  - 95% varianza: {n_componentes_95} componentes")
print(f"  - 99% varianza: {n_componentes_99} componentes")
print(f"\nDimensionalidad original: {X_escalado.shape[1]} características")
print(f"Dimensionalidad reducida: {n_componentes_95} componentes")
print(f"Reducción: {(1 - n_componentes_95/X_escalado.shape[1])*100:.1f}%")

# Usar el número de componentes para 95% de varianza
n_componentes_seleccionado = n_componentes_95

### 3.4 Transformación de Datos con PCA

In [ ]:
# Aplicar PCA con el número de componentes seleccionado
pca = PCA(n_components=n_componentes_seleccionado, random_state=SEED)
X_pca = pca.fit_transform(X_escalado)

# Convertir a DataFrame para facilitar el manejo
columnas_pca = [f'PC{i+1}' for i in range(n_componentes_seleccionado)]
X_pca_df = pd.DataFrame(X_pca, columns=columnas_pca, index=X_escalado.index)

print(f"✓ PCA aplicado exitosamente.")
print(f"\nDimensiones de X después de PCA: {X_pca_df.shape}")
print(f"Varianza total explicada: {pca.explained_variance_ratio_.sum()*100:.2f}%")

### 3.5 Visualización de Datos en 2D usando PCA

Vamos a visualizar los datos proyectados en las dos primeras componentes principales, coloreados por clase. Esto nos ayuda a:
- Entender la separabilidad de las clases en el espacio reducido
- Identificar posibles solapamientos entre clases
- Validar visualmente que PCA mantiene la estructura de los datos

In [ ]:
# Crear el gráfico de dispersión 2D
plt.figure(figsize=(12, 8))

# Graficar cada clase con un color diferente
for i, clase in enumerate(clases_originales):
    # Filtrar puntos de esta clase
    mascara = (y_codificado == i)
    plt.scatter(X_pca_df.loc[mascara, 'PC1'], 
               X_pca_df.loc[mascara, 'PC2'],
               label=clase,
               alpha=0.6,
               s=20)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% varianza)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% varianza)', fontsize=12)
plt.title('Proyección de Datos en las Primeras Dos Componentes Principales', fontsize=14, fontweight='bold')
plt.legend(title='Clase', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Observación:")
print("En el gráfico podemos observar cómo se distribuyen las diferentes clases en el espacio")
print("de las dos primeras componentes principales. Una buena separación visual indica que")
print("los modelos de clasificación deberían poder distinguir entre las clases.")

### 3.6 Análisis de Componentes Principales (Opcional)

Podemos analizar qué características originales contribuyen más a cada componente principal.

In [ ]:
# Crear DataFrame con los componentes (loadings) de PCA
componentes_df = pd.DataFrame(
    pca.components_.T,
    columns=columnas_pca,
    index=X_escalado.columns
)

print("Contribución de cada característica original a las componentes principales:")
print("="*80)
print(componentes_df.round(3))

# Visualizar las contribuciones de las primeras 3 componentes
fig, axes = plt.subplots(1, min(3, n_componentes_seleccionado), figsize=(18, 5))
if n_componentes_seleccionado == 1:
    axes = [axes]

for i in range(min(3, n_componentes_seleccionado)):
    componentes_df[f'PC{i+1}'].plot(kind='barh', ax=axes[i])
    axes[i].set_title(f'Contribuciones a PC{i+1}\n({pca.explained_variance_ratio_[i]*100:.1f}% varianza)', 
                     fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Peso', fontsize=10)
    axes[i].axvline(x=0, color='k', linestyle='-', linewidth=0.8)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.7 Resumen de Reducción de Dimensionalidad

**Logros de PCA:**

1. ✓ Reducción significativa de dimensionalidad manteniendo ~95% de la varianza
2. ✓ Eliminación de redundancia entre características correlacionadas
3. ✓ Visualización exitosa de los datos en 2D
4. ✓ Identificación de las características más importantes para cada componente

**Datos disponibles:**
- `X_escalado`: Datos originales escalados (todas las características)
- `X_pca_df`: Datos transformados con PCA (dimensionalidad reducida)

**Decisión:** Usaremos `X_pca_df` para entrenar los modelos, ya que esto mejorará la eficiencia computacional y podría mejorar el desempeño al reducir el ruido.

---